# Titanic classification model

Using StatQuest video on decision forests: clustering for missing data, n optimization, etc

(video 1)[https://youtu.be/J4Wdy0Wc_xQ?si=L08xHb3JQvmHEHjj]

(video 2)[https://youtu.be/sQ870aTKqiM?si=tXDFxqsgornauspB]

In [2]:
import numpy as np # linear algebra
import pandas as pd
import ydf

In [15]:
train_data = pd.read_csv("./train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
test_data = pd.read_csv("./test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [13]:
summary = pd.DataFrame({
    'Column Type': train_data.dtypes,
    'Missing Values': train_data.isnull().sum(),
    'Total': train_data.count(),
    'Uniques': train_data.nunique(),
    'min': train_data.min(numeric_only=True),
    'mean': train_data.mean(numeric_only=True),
    'max': train_data.max(numeric_only=True)
}).reset_index().rename(columns={'index': 'Column'})

print("Train data summary:\n")
print(summary)

Train data summary:

         Column Column Type  Missing Values  Total  Uniques   min        mean       max
0           Age     float64             177    714       88  0.42   29.699118   80.0000
1         Cabin      object             687    204      147   NaN         NaN       NaN
2      Embarked      object               2    889        3   NaN         NaN       NaN
3          Fare     float64               0    891      248  0.00   32.204208  512.3292
4          Name      object               0    891      891   NaN         NaN       NaN
5         Parch       int64               0    891        7  0.00    0.381594    6.0000
6   PassengerId       int64               0    891      891  1.00  446.000000  891.0000
7        Pclass       int64               0    891        3  1.00    2.308642    3.0000
8           Sex      object               0    891        2   NaN         NaN       NaN
9         SibSp       int64               0    891        7  0.00    0.523008    8.0000
10     Surv

Based on above, filling Cabin nan's with an empty string 

In [14]:
summary = pd.DataFrame({
    'Column Type': test_data.dtypes,
    'Missing Values': test_data.isnull().sum(),
    'Total': test_data.count(),
    'Uniques': test_data.nunique(),
    'min': test_data.min(numeric_only=True),
    'mean': test_data.mean(numeric_only=True),
    'max': test_data.max(numeric_only=True)
}).reset_index().rename(columns={'index': 'Column'})

print("Test data summary:\n")
print(summary)

Test data summary:

         Column Column Type  Missing Values  Total  Uniques     min         mean        max
0           Age     float64              86    332       79    0.17    30.272590    76.0000
1         Cabin      object             327     91       76     NaN          NaN        NaN
2      Embarked      object               0    418        3     NaN          NaN        NaN
3          Fare     float64               1    417      169    0.00    35.627188   512.3292
4          Name      object               0    418      418     NaN          NaN        NaN
5         Parch       int64               0    418        8    0.00     0.392344     9.0000
6   PassengerId       int64               0    418      418  892.00  1100.500000  1309.0000
7        Pclass       int64               0    418        3    1.00     2.265550     3.0000
8           Sex      object               0    418        2     NaN          NaN        NaN
9         SibSp       int64               0    418        7 

In [16]:
model = ydf.RandomForestLearner(label="Survived",
                                task=ydf.Task.CLASSIFICATION).train(train_data)

Train model on 891 examples
Model trained in 0:00:00.118915


In [25]:
# model.label_classes() # run to confirm order for np.where()
y_prob = model.predict(test_data)
y_pred = np.where(y_prob <= 0.5, 0, 1)

In [26]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_pred})
output.to_csv('submission_ydf_v0.csv', index=False)

## Results:

Score: 0.78229

Default ydf model used

Described model below

In [27]:
model.describe()